The initial template for this code was provided by the professor Giovanni Squillero `<giovanni.squillero@polito.it>`. All copyrights **`(c)`** are reserved to him.
The original code can be found at https://github.com/squillero/computational-intelligence

In [795]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq




In [796]:
PUZZLE_DIM =3
action = namedtuple('Action', ['pos1', 'pos2'])

In [797]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


In [798]:
RANDOMIZE_STEPS = 100_000
target_state=np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
start_state = target_state
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    start_state = do_action(start_state, choice(available_actions(start_state)))

# start_state = np.array([np.array([0,2,1]),np.array([3,7,5]),np.array([8,6,4])])

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

In [799]:
#Coords of target matrix. They are used to compute manhattan distance.
x_target=np.zeros(PUZZLE_DIM**2)
y_target=np.zeros(PUZZLE_DIM**2)
for i in range(1,PUZZLE_DIM**2):
    pos = np.where(target_state == i)
    x_target[i], y_target[i] = pos[0][0], pos[1][0] 

class Position: #The position class represent the state of the game with the f (h+g) embedded.
    def __init__(self,grid:np.array,g:int,prev_pos=None) -> None:
        self.grid=grid
        self.prev_pos=prev_pos
        self.g=g
        self.f=int(self.__h()+g)
    def __h(self) -> int:
        #This is the heuristic: MANHATTAN DISTANCE over every element
        tot_dist=0
        for i in range(1,PUZZLE_DIM**2):
            x_2,y_2=np.where(self.grid==i)
            tot_dist+=abs(x_target[i]-x_2.item())+abs(y_target[i]-y_2.item())
        return tot_dist
    

    
    #For heap
    def __lt__(self, other):
     return self.f<other.f
    

    def __hash__(self):
            return hash(self.grid.tobytes()) #note that hash do not depend on prev_pos
    
    def __eq__(self, other):
        return np.array_equal(self.grid, other.grid) 






In [800]:

def a_star(start_pos:Position):

    curr_pos=start_pos
    closed_set = set() #Visited Positions
    priority_heap=[] #Priority queue of open_set elements
    # open_set={hash(start_pos) : start_pos.g} 

    while True :
        # open_set.pop(hash(curr_pos))
        closed_set.add(hash(curr_pos)) 
        # if((curr_pos.grid==target_state).all()):
        if(curr_pos.f-curr_pos.g==0):
            print("Elems open list: ", len(priority_heap))
            print("Evaluated pos (cost): ",len(closed_set)-1) #-1 because of starting position
            return curr_pos
  
        new_actions=available_actions(curr_pos.grid)
        for action in new_actions:
            new_pos=Position(do_action(curr_pos.grid,action),curr_pos.g+1,curr_pos)
            if hash(new_pos) in closed_set: #Avoid adding to heap already evaluated positions.
                continue
            # elif hash(new_pos) not in open_set or new_pos.g < open_set[hash(new_pos) ]: 
                # open_set[hash(new_pos) ] = new_pos.g
            heapq.heappush(priority_heap, new_pos) 
           
            
        while hash(curr_pos) in closed_set: #if curr_pos is in closed_set means that we found a way to reach the same grid with a lower g, so we just pop it and look for another Position
            curr_pos=heapq.heappop(priority_heap) 


        # if(evaluated_pos%50000==0):
        #     ic(curr_pos.grid)

start_pos=Position(start_state,0)


print("Start grid:\n",start_pos.grid,"\nStart heuristic: ",start_pos.f)

found_sol=a_star(start_pos)   


print("Steps: ",found_sol.g)
print("Found solution (backwards): ")
while True:
    print(found_sol.grid,"\n ")
    if(found_sol.prev_pos is None):
        break
    found_sol=found_sol.prev_pos




Start grid:
 [[8 6 5]
 [4 0 7]
 [2 3 1]] 
Start heuristic:  20
Elems open list:  1010
Evaluated pos (cost):  1677
Steps:  26
Found solution (backwards): 
[[1 2 3]
 [4 5 6]
 [7 8 0]] 
 
[[1 2 3]
 [4 5 0]
 [7 8 6]] 
 
[[1 2 3]
 [4 0 5]
 [7 8 6]] 
 
[[1 0 3]
 [4 2 5]
 [7 8 6]] 
 
[[0 1 3]
 [4 2 5]
 [7 8 6]] 
 
[[4 1 3]
 [0 2 5]
 [7 8 6]] 
 
[[4 1 3]
 [2 0 5]
 [7 8 6]] 
 
[[4 1 3]
 [2 5 0]
 [7 8 6]] 
 
[[4 1 0]
 [2 5 3]
 [7 8 6]] 
 
[[4 0 1]
 [2 5 3]
 [7 8 6]] 
 
[[4 5 1]
 [2 0 3]
 [7 8 6]] 
 
[[4 5 1]
 [2 8 3]
 [7 0 6]] 
 
[[4 5 1]
 [2 8 3]
 [7 6 0]] 
 
[[4 5 1]
 [2 8 0]
 [7 6 3]] 
 
[[4 5 0]
 [2 8 1]
 [7 6 3]] 
 
[[4 0 5]
 [2 8 1]
 [7 6 3]] 
 
[[4 8 5]
 [2 0 1]
 [7 6 3]] 
 
[[4 8 5]
 [2 6 1]
 [7 0 3]] 
 
[[4 8 5]
 [2 6 1]
 [0 7 3]] 
 
[[4 8 5]
 [0 6 1]
 [2 7 3]] 
 
[[0 8 5]
 [4 6 1]
 [2 7 3]] 
 
[[8 0 5]
 [4 6 1]
 [2 7 3]] 
 
[[8 6 5]
 [4 0 1]
 [2 7 3]] 
 
[[8 6 5]
 [4 7 1]
 [2 0 3]] 
 
[[8 6 5]
 [4 7 1]
 [2 3 0]] 
 
[[8 6 5]
 [4 7 0]
 [2 3 1]] 
 
[[8 6 5]
 [4 0 7]
 [2 3 1]] 
 
